<a href="https://colab.research.google.com/github/zjzsu2000/CMPE258/blob/master/Graded%20Assignment1-%20Do%20MNIST%20classifier/MNIST_classifier_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Do MNIST classifier using numpy and python 

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

## Import the dataset:MNIST database of handwritten digits

Dataset of 60,000 28x28 grayscale images of the 10 digits, along with a test set of 10,000 images.

https://keras.io/datasets/#mnist-database-of-handwritten-digits

In [0]:
from tensorflow.keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [0]:
train_labels

In [0]:
np.int8(test_labels)

In [0]:
class_names = ['0', '1', '2', '3', '4',
               '5', '6', '7', '8', '9']

## Explore the data

Dataset of 60,000 28x28 grayscale images of the 10 digits, along with a test set of 10,000 images.

In [0]:
train_images.shape

Likewise, there are 60,000 labels in the training set:

In [0]:
len(train_labels)

Each label is an integer between 0 and 9:

In [0]:
train_labels

There are 10,000 images in the test set. Again, each image is represented as 28*28 pixels:

In [0]:
test_images.shape

And the test set contains 10,000 images labels:

In [0]:
len(test_labels)

## Preprocess the data
#Part F:Scaling the input 
# scaling the input to 255

In [0]:
plt.figure()
plt.imshow(train_images[0])
plt.colorbar()
plt.grid(False)
plt.show()

Scale these values to a range of 0 to 1 before feeding them to the neural network model. 

In [0]:
np.random.seed(1)

In [0]:
images, labels = (train_images[0:5000].reshape(5000,28*28) / 255), train_labels[0:5000]

In [0]:
one_hot_labels = np.zeros((len(labels), 10))

# assigning 1 to where the label would be for that particular array
# for example, if the array 
for i,l in enumerate(labels):
  one_hot_labels[i][l] = 1
labels = one_hot_labels

import sys, numpy as np

test_img = test_images.reshape(len(test_images), 28*28) / 255
test_label = np.zeros((len(test_labels), 10))

for i,l in enumerate(test_labels):
    test_label[i][l] = 1

In [0]:
images.shape

In [0]:
labels

To verify that the data is in the correct format and ready to build and train the network, let's display the first 25 images from the *training set* and display the class name below each image.

In [0]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

#Using the numpy to def the functions

In [0]:
import numpy as np
np.random.seed(1)

In [0]:
 #return x if x > 0, else return 0
def relu(x):
      return (x>=0)*x

# returns 1 if input > 0, else return 0
def relu2deriv(r1Output):
      return r1Output>=0

* Part A:Mini batch 
* Part B:Random weight initializations 
* Part C:Data Augmentation 
* Part D:Using 3 or more layers for training 
* Part E:Using Relu activation function
* Part G:Using appropriate learning rate
* Part H:Testing and training accuracy (plotting results with confusion matrix)



In [0]:
from keras.preprocessing.image import ImageDataGenerator

In [0]:
train_images = train_images.reshape(-1,28,28,1) # reshape the train_images dataset in 3 dimensions

In [0]:
train_images.shape

In [0]:
datagenerator = ImageDataGenerator(featurewise_center=False,samplewise_center=False, featurewise_std_normalization=False, 
                                   samplewise_std_normalization=False, zca_whitening=False, 
                                   rotation_range=10, zoom_range = 0.1, width_shift_range=0.2, 
                                    height_shift_range=0.1, horizontal_flip=False, vertical_flip=False) 

In [0]:
datagenerator.fit(train_images)

In [0]:
from matplotlib import pyplot

for X_batch, y_batch in datagenerator.flow(train_images, train_labels, batch_size=9):
	# create a grid of 3x3 images
	for i in range(0, 9):
		pyplot.subplot(330 + 1 + i)
		pyplot.imshow(X_batch[i].reshape(28, 28), cmap=pyplot.get_cmap('gray'))

	pyplot.show()
	break

In [0]:
batchSize=100 # batch size
alpha=0.001 # learning rate
iterations=300 # number of iterations
pixelsPerImage=28*28 # pixels per image
numLabels=10 # number of labels
hiddenSize=100 # hidden size
weights_0_1 = 0.2 * np.random.random((pixelsPerImage, hiddenSize)) - 0.1 # weights from layer 0 to 1
weights_1_2=0.2*np.random.random((hiddenSize,numLabels))-0.1 # weights from layer 1 to 2

In [0]:
testErrList = np.array([])
testAccList = np.array([])
trainErrList = np.array([])
trainAccList = np.array([])

In [0]:
for i in range(iterations):
  error,correctCnt=(0.0, 0)
  for j in range(int(len(images)/batchSize)):
    batchStart,batchEnd=((j*batchSize),((j+1)*batchSize))
    layer_0=images[batchStart:batchEnd]
    layer_1=relu(np.dot(layer_0,weights_0_1))
    dropoutMask=np.random.randint(2,size=layer_1.shape)
    layer_2=np.dot(layer_1,weights_1_2)
    error+=np.sum((labels[batchStart:batchEnd]-layer_2)**2)
    for k in range(batchSize):
      # counting # of the correct predicts
      correctCnt+=int(np.argmax(layer_2[k:k+1])==np.argmax(labels[batchStart+k:batchStart+k+1]))          
      # difference predicted and actual 
      layer_2_delta = (labels[batchStart:batchEnd] - layer_2) / batchSize
      # backprop & adjust weights then add dropout mask
      layer_1_delta = layer_2_delta.dot(weights_1_2.T) * relu2deriv(layer_1)
      layer_1_delta *= dropoutMask  
      weights_1_2 += alpha * layer_1.T.dot(layer_2_delta)
      weights_0_1 += alpha * layer_0.T.dot(layer_1_delta)        
    
  if (i%10 == 0):
    testError, testCorrectCnt= (0.0, 0)
    for j in range(len(test_img)):
      layer_0 = test_img[j:j+1]
      layer_1 = relu(np.dot(layer_0, weights_0_1))
      layer_2 = np.dot(layer_1, weights_1_2)
      testError += np.sum((test_label[j:j+1] - layer_2) ** 2)
      testCorrectCnt += int(np.argmax(layer_2) == np.argmax(test_label[j:j+1]))
     
    testErrList = np.append(testErrList, [testError/ float(len(test_img))])
    testAccList = np.append(testAccList, [testCorrectCnt / float(len(test_img))])
    trainErrList = np.append(trainErrList, [error/float(len(images))])
    trainAccList = np.append(trainAccList, [correctCnt/float(len(images))])
    print("\n Iamge:"+str(i)+ " Testing Error:" + str(testError/ float(len(test_img)))[0:5] +\
                     " Testing Accuracy:" + str(testCorrectCnt/ float(len(test_img)))+\
                     " Training Error:" + str(error/float(len(images)))[0:5] +\
                     " Training Accuracy:" + str(correctCnt/float(len(images))))

#plot the result

In [0]:
import matplotlib.pyplot as plt

plt.title('Training Result Plotting\n(Yellow for Accuracy&Blue for Error)')
plt.plot(trainAccList, color='y')
plt.plot(trainErrList, color='b')
plt.show()



In [0]:
plt.title('Testing Result Plotting\n(Yellow for Accuracy&Blue for Error)')
plt.plot(testAccList, color='y')
plt.plot(testErrList, color='b')
plt.show()


In [0]:
plt.title('Training & Testing Result Plotting\n(Yellow for TrainingAccuracy&Blue for TestingError)')
plt.plot(trainAccList, color='y')
plt.plot(testAccList, color='b')
plt.show()

# Confusion matrix 


In [0]:
from sklearn.metrics import confusion_matrix

Predict the values from the test_img(validation dataset)


In [0]:
y_pred = np.array([])
for i in range(len(test_img)):
  layer_0 = test_img[i:i+1]
  layer_1 = relu(np.dot(layer_0, weights_0_1))
  layer_2 = np.dot(layer_1, weights_1_2)
  y_pred = np.append(y_pred, np.argmax(layer_2))

In [0]:
y_test=test_labels

In [0]:
# compute the confusion matrix
confusionMatrix=confusion_matrix(y_test, y_pred) 

In [0]:
confusionMatrix

In [0]:
classes=range(10)
cmap=plt.cm.Reds
plt.imshow(confusionMatrix, interpolation='nearest', cmap=cmap)
plt.title('Confusion matrix')
plt.colorbar()
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation=45)
plt.yticks(tick_marks, classes)
thres = confusionMatrix.max() / 2.
import itertools
for i, j in itertools.product(range(confusionMatrix.shape[0]), range(confusionMatrix.shape[1])):
  plt.text(j, i, confusionMatrix[i, j], horizontalalignment="center", color="white" if confusionMatrix[i, j] > thres else "blue")
  #plt.tight_layout()
  plt.ylabel('True')
  plt.xlabel('Predicted')

## Errors 


In [0]:
errors = (y_pred-y_test!=0)

yPredErrors = y_pred[errors]
yTestErrors = y_test[errors]


In [0]:
yPredErrorsProb = np.max(yPredErrors)
print(yPredErrorsProb)

In [0]:
TPErrors = np.take(yPredErrors, yTestErrors)
print(TPErrors)

In [0]:
deltaPredTrueErrors = yPredErrorsProb - TPErrors
print(deltaPredTrueErrors)

Sorted it

In [0]:
deltaPredTrueErrors=np.argsort(deltaPredTrueErrors)
print(deltaPredTrueErrors)

In [0]:
topErrors = deltaPredTrueErrors[-8:]


# Show the top 8 errors

In [0]:
imageErrors = test_img[errors]

In [0]:
n = 0
fig, ax = plt.subplots(2,4,sharex=True,sharey=True)
for row in range(2):
  for col in range(4):
    error = topErrors[n]
    ax[row,col].imshow((imageErrors[error]).reshape((28,28)))
    ax[row,col].set_title("Predicted :{}\nTrue :{}".format(yPredErrors[error],yTestErrors[error]))
    n += 1